# Importe das bibliotecas

In [111]:
import pandas as pd
from nltk.corpus import stopwords
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords 
from nltk.stem.lancaster import LancasterStemmer
import re
from nltk.stem import PorterStemmer
from textblob import Word
import numpy as np

## Lendo os dados
##### leitura do arquivo de treinamento do conjunto de dados

In [103]:
train = pd.read_csv('imdb-reviews-pt-br.csv').drop('sentiment', axis=1)
train2 = pd.read_csv('imdb-reviews-pt-br.csv').drop('sentiment', axis=1)

In [60]:
train.head()

,id,text_en,text_pt
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...


## Analises básicas

### Quantidade de palavras
##### A intuição básica por trás disso é que, geralmente, os sentimentos negativos contêm uma quantidade menor de palavras do que as positivas.

In [42]:
train['word_count'] = train['text_en'].apply(lambda x: len(str(x).split(" ")))
train[['text_en','word_count']].head()

,text_en,word_count
0,Once again Mr. Costner has dragged out a movie...,168
1,This is an example of why the majority of acti...,232
2,"First of all I hate those moronic rappers, who...",215
3,Not even the Beatles could write songs everyon...,308
4,Brass pictures movies is not a fitting word fo...,238


### Quantidade de caracteres
##### Esse recurso também é baseado na intuição do recurso anterior. 

In [43]:
train['char_count'] = train['text_en'].str.len() ## this also includes spaces
train[['text_en','char_count']].head()

,text_en,char_count
0,Once again Mr. Costner has dragged out a movie...,897
1,This is an example of why the majority of acti...,1305
2,"First of all I hate those moronic rappers, who...",1207
3,Not even the Beatles could write songs everyon...,1664
4,Brass pictures movies is not a fitting word fo...,1444


### Comprimento Médio da Palavra

In [44]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['text_en'].apply(lambda x: avg_word(x))
train[['text_en','avg_word']].head()

,text_en,avg_word
0,Once again Mr. Costner has dragged out a movie...,4.345238
1,This is an example of why the majority of acti...,4.629310
2,"First of all I hate those moronic rappers, who...",4.618605
3,Not even the Beatles could write songs everyon...,4.405844
4,Brass pictures movies is not a fitting word fo...,5.071429


### Número de Stopwords

In [47]:
stop = stopwords.words('english')

train['stopwords'] = train['text_en'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['text_en','stopwords']].head()

,text_en,stopwords
0,Once again Mr. Costner has dragged out a movie...,81
1,This is an example of why the majority of acti...,93
2,"First of all I hate those moronic rappers, who...",97
3,Not even the Beatles could write songs everyon...,127
4,Brass pictures movies is not a fitting word fo...,86


### Número de Caracteres especiais
##### calcular o número de hashtags ou menções presentes nele. 
##### uso da função "startswith" para achar "@" e "#" 

In [48]:
train['hastags'] = train['text_en'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['text_en','hastags']].head()

,text_en,hastags
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures movies is not a fitting word fo...,0


### Número de palavras Uppercase
##### 'Anger' ou 'Rage' são geralmente escritas em maiúsculas

In [49]:
train['upper'] = train['text_en'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
train[['text_en','upper']].head()

,text_en,upper
0,Once again Mr. Costner has dragged out a movie...,3
1,This is an example of why the majority of acti...,1
2,"First of all I hate those moronic rappers, who...",2
3,Not even the Beatles could write songs everyon...,2
4,Brass pictures movies is not a fitting word fo...,3



# Pré processamento Básico
##### Antes de realizar estração de textos e recursos, deve-se limpar os dados para obter melhores recursos


### Lower case
##### A primeira etapa de pré-processamento que faremos é transformar nossos tweets em minúsculas.

In [61]:
train['text_en'] = train['text_en'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train.head()

,id,text_en,text_pt
0,1,once again mr. costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,this is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...
2,3,"first of all i hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,not even the beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...


### Remover pontuações
##### Remover a pontuação, pois não adiciona informações extras ao tratar dados de texto.
##### Toda a pontuação, incluindo "#" e "@", foi removida dos dados de treinamento

In [63]:
train['text_en'] = train['text_en'].str.replace('[^\w\s]','')
train.head()

,id,text_en,text_pt
0,1,once again mr costner has dragged out a movie ...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,this is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...
2,3,first of all i hate those moronic rappers who ...,"Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,not even the beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...


### Correção Ortográfica
##### A escrita correta das palavras ajuda na eliminação de palavras semelhantes

In [68]:
train['text_en'] = train['text_en'].apply(lambda x: str(TextBlob(x).correct()))
train.head()

,id,text_en,text_pt
0,1,once again mr coster has dragged out a movie f...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,this is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...
2,3,first of all i hate those chronic trappers who...,"Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,not even the beetles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,brass pictures moves is not a fitting word for...,Filmes de fotos de latão não é uma palavra apr...


### Remover Stopwords
##### Assumimos que palavras que se repetem muitas vezes no dataset, são descartaveis, por serem provavelmente palavras comuns, logo devem ser retiradas do dataset para diminuir a dimencionalidade do mesmo

In [70]:
stop = stopwords.words('english')
train['text_en'] = train['text_en'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train.head()

,id,text_en,text_pt
0,1,mr coster dragged movie far longer necessary a...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,example majority action films genetic boring r...,Este é um exemplo do motivo pelo qual a maiori...
2,3,first hate chronic trappers act gun pressed fo...,"Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,even beetles could write songs everyone liked ...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,brass pictures moves fitting word really somew...,Filmes de fotos de latão não é uma palavra apr...


##### Além da remoção de palavras comuns na lingua, é aconcelhavel remoção de palavras comuns no dataset de forma geral

In [72]:
freq = pd.Series(' '.join(train['text_en']).split()).value_counts()[:10]
print(freq)
freq = list(freq.index)
train['text_en'] = train['text_en'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train.head()

the     617933
a       304724
and     304193
of      273164
to      252518
is      200638
in      173520
it      145851
i       141937
this    141050
dtype: int64


,id,text_en,text_pt
0,1,mr coster dragged movie far longer necessary a...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,example majority action films genetic boring r...,Este é um exemplo do motivo pelo qual a maiori...
2,3,first hate chronic trappers act gun pressed fo...,"Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,even beetles could write songs everyone liked ...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,brass pictures moves fitting word really somew...,Filmes de fotos de latão não é uma palavra apr...


### Remoção de palavras raras
##### Palavras com poucas aparições, pois trata-se de ruído nos dados

In [74]:
freq = pd.Series(' '.join(train['text_en']).split()).value_counts()[-10:]
print(freq)
freq = list(freq.index)
train['text_en'] = train['text_en'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train.head()

kittytype          1
tanaaz             1
boredomill         1
kidsrivers         1
laughinglive       1
dontlive           1
anewyeah           1
vacuousthis        1
arrrrghdon         1
makingvolckmans    1
dtype: int64


,id,text_en,text_pt
0,1,mr coster dragged movie far longer necessary a...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,example majority action films genetic boring r...,Este é um exemplo do motivo pelo qual a maiori...
2,3,first hate chronic trappers act gun pressed fo...,"Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,even beetles could write songs everyone liked ...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,brass pictures moves fitting word really somew...,Filmes de fotos de latão não é uma palavra apr...


### Demais remoções

* Remoção de usernames
* Remoção de emails
* Remoção de Urls
* Reoção de digitos
* Remoção de caracteres repetidos

In [ ]:
train['text_en'] = train['text_en'].apply(lambda x: re.sub(r'@[\w\.-]+','',x))
train['text_en'] = train['text_en'].apply(lambda x: re.sub(r'[\w\.-]+@[\w\.-]+','',x))
train['text_en'] = train['text_en'].apply(lambda x: re.sub(r'^https?:\/\/.*[\r\n]*','',x))
train['text_en'] = train['text_en'].apply(lambda x: re.sub(r'[0-9]','',x))
train['text_en'] = train['text_en'].apply(lambda x: re.sub(r"(\w)\1*", r'\1', x))

# Advance Text Processing

### Tokenization
##### Deve-se ter em mente que as palavras são frequentemente usadas em sua forma abreviada. Por exemplo, "your" é usado como "ur"

In [77]:
train['text_en'] = train['text_en'].apply(lambda x: nltk.word_tokenize(x))
train.head()

,id,text_en,text_pt
0,1,"[mr, coster, dragged, movie, far, longer, nece...","Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,"[example, majority, action, films, genetic, bo...",Este é um exemplo do motivo pelo qual a maiori...
2,3,"[first, hate, chronic, trappers, act, gun, pre...","Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,"[even, beetles, could, write, songs, everyone,...",Nem mesmo os Beatles puderam escrever músicas ...
4,5,"[brass, pictures, moves, fitting, word, really...",Filmes de fotos de latão não é uma palavra apr...


### Stemming
##### Trata-se de deixar apenas o radical da palavra

In [79]:
st = PorterStemmer()
train['text_en'][:5].apply(lambda x: " ".join([st.stem(word) for word in x]))

0    mr coster drag movi far longer necessari asid ...
1    exampl major action film genet bore realli not...
2    first hate chronic trapper act gun press foreh...
3    even beetl could write song everyon like altho...
4    brass pictur move fit word realli somewhat bra...
Name: text_en, dtype: object

### Lemmatization
##### Solução mais eficiênte que o Stemming, pois o espaço amostral dos dados não fica reduzido demais, evitando que palavras diferentes com o mesmo sufixo sujem os dados

In [81]:
train['text_en'] = train['text_en'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))
train.head()

,id,text_en,text_pt
0,1,mr coster dragged movie far longer necessary a...,"Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,example majority action film genetic boring re...,Este é um exemplo do motivo pelo qual a maiori...
2,3,first hate chronic trapper act gun pressed for...,"Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,even beetle could write song everyone liked al...,Nem mesmo os Beatles puderam escrever músicas ...
4,5,brass picture move fitting word really somewha...,Filmes de fotos de latão não é uma palavra apr...


### N-grams
##### N-gramas são a combinação de múltiplas palavras usadas juntas (unigramas, bigramas, trigramas)
##### O princípio básico por trás dos n-grams é que eles capturam a estrutura da linguagem, como qual letra ou palavra provavelmente seguirá. Quanto maior o n-grama, mais contexto tem-se que trabalhar. Se os seus n-grams forem muito curtos, pode não conseguir capturar diferenças importantes. Por outro lado, se eles são muito longos, pode deixar de capturar o “conhecimento geral” e apenas se ater a casos particulares.

In [90]:
train['text_en'] = train['text_en'].apply(lambda x: TextBlob(x).ngrams(2))
train.head()

,id,text_en,text_pt
0,1,"[[mr, coster], [coster, dragged], [dragged, mo...","Mais uma vez, o Sr. Costner arrumou um filme p..."
1,2,"[[example, majority], [majority, action], [act...",Este é um exemplo do motivo pelo qual a maiori...
2,3,"[[first, hate], [hate, chronic], [chronic, tra...","Primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,"[[even, beetle], [beetle, could], [could, writ...",Nem mesmo os Beatles puderam escrever músicas ...
4,5,"[[brass, picture], [picture, move], [move, fit...",Filmes de fotos de latão não é uma palavra apr...


### Term Frequency
##### Frequência de termo é simplesmente a razão entre a contagem de uma palavra presente em uma frase e a duração da sentença.
#### TF = (Número de vezes que o termo T aparece na linha em particular) / (número de termos nessa linha)

In [122]:
tf1 = (train2['text_en'][1:2]).apply(lambda x: pd.value_counts(x.split())).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,the,13
1,and,8
2,this,6
3,of,5
4,better,3
5,was,3
6,which,3
7,or,3
8,is,3
9,film,3


### Inverse Document Frequency
##### Inverse Document Frequency (IDF) é que uma palavra não é muito útil para nós se estiver aparecendo em todos os documentos.
##### Portanto, o IDF de cada palavra é o log da proporção do número total de linhas para o número de linhas em que essa palavra está presente.
#### IDF = log (N / n)
##### Onde, N é o número total de linhas en é o número de linhas em que a palavra estava presente.

In [125]:
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(train2.shape[0]/(len(train2[train2['text_en'].str.contains(word, na=False)])))

tf1

,words,tf,idf
0,the,13,0.006939
1,and,8,0.029422
2,this,6,0.173573
3,of,5,0.045945
4,better,3,1.715205
5,was,3,0.396217
6,which,3,1.249805
7,or,3,0.013783
8,is,3,0.005433
9,film,3,0.521851


### Term Frequency – Inverse Document Frequency (TF-IDF)
##### TF-IDF é a multiplicação do TF e IDF, penalizando as palavras que mais se repetem

In [127]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,the,13,0.006939,0.090205
1,and,8,0.029422,0.235379
2,this,6,0.173573,1.041437
3,of,5,0.045945,0.229727
4,better,3,1.715205,5.145615
5,was,3,0.396217,1.188652
6,which,3,1.249805,3.749416
7,or,3,0.013783,0.041348
8,is,3,0.005433,0.016300
9,film,3,0.521851,1.565554


In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(train2['text_en'])

train_vect

<49459x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2232210 stored elements in Compressed Sparse Row format>

### Bag of Words
##### Bag of Words (BoW) refere-se à representação de texto que descreve a presença de palavras nos dados do texto. A intuição por trás disso é que dois campos de texto semelhantes conterão palavras semelhantes e, portanto, terão um pacote similar de palavras. Além disso, somente a partir do texto podemos aprender algo sobre o significado do documento.

In [132]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train2['text_en'])
train_bow

<49459x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 4345747 stored elements in Compressed Sparse Row format>

### Word Embeddings
##### Word Embeddings é a representação de texto na forma de vetores. A ideia subjacente aqui é que palavras semelhantes terão uma distância mínima entre seus vetores.
##### Os modelos Word2Vec exigem muito texto